In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from sklearn.metrics import f1_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [6]:
data = pd.read_csv('./Data/weatherAUS.csv')

In [7]:
data = data.fillna(-1)

data = data.dropna(how='all')

In [8]:
le = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column].astype(str))

In [9]:
scaler = MinMaxScaler()
for column in data.columns:
    if data[column].dtype == np.float64:
        data[column] = scaler.fit_transform(data[column].values.reshape(-1, 1))

In [10]:
X = data.drop('RainTomorrow', axis=1)
y = data['RainTomorrow']

In [11]:
n_splits = 10
kf = KFold(n_splits=n_splits)

In [19]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(1, activation='sigmoid')
])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [1]:
fold_accuracy = []
fold_f1 = []
fold_recall = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Trenuj model na danych treningowych
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    
    # Predykcja na danych testowych
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)  # Konwertuj wyniki na klasy binarne (0 lub 1)
    
    # Ocena modelu na danych testowych
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    fold_accuracy.append(accuracy)
    
    # Oblicz dodatkowe metryki
    f1 = f1_score(y_test, y_pred, average='micro')  # Zmiana ustawienia na 'micro'
    recall = recall_score(y_test, y_pred, average='micro')  # Zmiana ustawienia na 'micro'
    
    fold_f1.append(f1)
    fold_recall.append(recall)

NameError: name 'kf' is not defined

In [17]:
mean_accuracy = np.mean(fold_accuracy)
std_accuracy = np.std(fold_accuracy)
mean_f1 = np.mean(fold_f1)
std_f1 = np.std(fold_f1)
mean_recall = np.mean(fold_recall)
std_recall = np.std(fold_recall)

print(f'Średnia dokładność: {mean_accuracy} ± {std_accuracy}')
print(f'Średnie F1-score: {mean_f1} ± {std_f1}')
print(f'Średnia czułość: {mean_recall} ± {std_recall}')

Średnia dokładność: 0.7583940625190735 ± 0.024376946869838552
Średnie F1-score: 0.7583940602227417 ± 0.024376951753523172
Średnia czułość: 0.7583940602227417 ± 0.024376951753523172
